In [10]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [137]:
#Mettre un pas du maillage qui divise la distance entre val_max et val_min.

def creer_maillage(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> tuple:
    """
    La fonction créer_maillage prend en entrée 6 float qui permettent de délémiter les bords du maillage et un pas.
    Le pas du maillage représente la distance séparant un point du prochain. 
    
    Elle renvoie un tuple de 3 arrays contenant eux-même N-array qui représentent toutes les combinaisons possibles
    discrétisant ainsi notre maillage en un  ensemble de points.
    """ 
    nb_points_entre_x_min_x_max = int((x_max - x_min) / (pas_du_maillage)) + 1
    nb_points_entre_y_min_y_max = int((y_max - y_min) / (pas_du_maillage)) + 1
    nb_points_entre_z_min_z_max = int((z_max - z_min) / (pas_du_maillage)) + 1

    axe_x = np.linspace(x_min,x_max,nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min,y_max,nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min,z_max,nb_points_entre_z_min_z_max)

    maillage = np.meshgrid(axe_x,axe_y,axe_z)

    return maillage

cote = 10
X,Y,Z = creer_maillage(-cote,cote,-cote,cote,-cote,cote,1)

In [129]:
def trouver_signe(x : float) -> str:
    if x >= 0:
        return '+'
    return '-'

In [142]:
#Eviter d'essayer de voir le plot pour un cube de plus de 50x50x50 points, temps de réponse trop long si existant tout court.

def show_maillage(X : np.array, Y : np.array, Z : np.array, afficher_plot : bool, afficher_array : bool, CHARGE = None):
    """
    La fonction show_maillage() prend en argument les trois matrices issues de la fonction creer_maillage() et permet de
    visualiser soit les matrices, soit la représentation de celles-ci dans un plot.
    """
    if afficher_plot:
        if CHARGE is not None:
            fig = go.Figure(data=[go.Scatter3d(
                x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                mode="markers",
                marker=dict(size=2,
                            color=CHARGE.flatten(),
                            colorscale="inferno",
                            cmin = CHARGE.min(),
                            cmax = CHARGE.max(),
                            opacity=0.8,
                            colorbar=dict(
                            title="Charge",  
                            ticks="outside",  
                            tickvals=[CHARGE.min(), CHARGE.max()],  
                            ticktext=[f"{CHARGE.min():{trouver_signe(CHARGE.min())}.2f}", f"{CHARGE.max():{trouver_signe(CHARGE.max())}.2f}"],  
                        )) 
            )])

            fig.update_layout(
                title="Visualisation du maillage",
                scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
            )
            fig.show()
        else:
            fig = go.Figure(data=[go.Scatter3d(
                x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                mode="markers",
                marker=dict(size=2, opacity=0.8) 
            )])

            fig.update_layout(
                title="Visualisation du maillage",
                scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
            )
            fig.show()

    if afficher_array:
        print(f"La première matrice ressemble à :\n\n{X},\n\n"
              f"La deuxième matrice ressemble à :\n\n{Y},\n\n"
              f"La dernière matrice ressemble à :\n\n{Z}")
        
    return None

show_maillage(X, Y, Z, True, False)


In [ ]:
#Il faut encore gérer le cas X == Y == Z

def find_valeurs_point(x : float, y : float, z : float, X : np.array, Y : np.array, Z : np.array) -> tuple:
    """
    Cette fonction permet en prenant 3 matrices X,Y,Z issues de la fonction creer_maillage() en attribut 
    et 3 points correspondant au point de l'espace considéré dont on veut connaître les valeurs associées à X,Y,Z.
    Si les trois matrices sont égales, alors elle renvoie la valeur associée au point considéré.
    """
    nb_points_x = np.shape(X[0])[0]
    nb_points_y = len(X)
    nb_points_z = np.shape(X[0])[1]
    
    val_min_x = X[0][0,0]
    val_max_x = X[0][nb_points_x-1,0]
    val_min_y = Y[0][0,0]
    val_max_y = Y[nb_points_y-1][0,0]
    val_min_z = Z[0][0,0]
    val_max_z = Z[0][0,nb_points_z-1]

    if (nb_points_x-1) != 0 and (nb_points_y-1) != 0 and (nb_points_z-1) != 0:
        pas_suivant_x = float((int(val_max_x - val_min_x))  / (nb_points_x - 1))
        pas_suivant_y = float((int(val_max_y - val_min_y))  / (nb_points_y - 1))
        pas_suivant_z = float((int(val_max_z - val_min_z))  / (nb_points_z - 1))
    else:
        raise ZeroDivisionError()
    
    val_indice_x = int((x - val_min_x) / pas_suivant_x)
    val_indice_y = int((y - val_min_y) / pas_suivant_y)
    val_indice_z = int((z - val_min_z) / pas_suivant_z)

    return float(X[val_indice_y][val_indice_x,val_indice_z]), float(Y[val_indice_y][val_indice_x,val_indice_z]), float(Z[val_indice_y][val_indice_x,val_indice_z])

print(find_valeurs_point(0,5,-5,X,Y,Z))



(0.0, 5.0, -5.0)


In [151]:
def calcul_charge_tout_point_maillage(X : np.array, Y : np.array, Z : np.array, fonction) -> tuple:
    """
    La fonction calcul_charge_tout_point_maillage() prend en entrée les 3 matrices qu'output la fonction
    creer_maillage() afin de calculer la charge portée par tous les points selon la fonction fournie.
    """
    return fonction(X,Y,Z)

a = calcul_charge_tout_point_maillage(X,Y,Z, lambda x,y,z : np.cos(x) + np.sin(y) + np.sin(z))

In [152]:
show_maillage(X,Y,Z,True,False,a)